In [107]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init
from torch.utils.data import DataLoader
import torch.optim as optim

In [108]:
device='cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
if device=='cuda':
    torch.cuda.manual_seed_all(777)

In [109]:
learning_rate=0.001
training_epochs=30
batch_size=32

In [110]:
cifar10_train = dsets.CIFAR10(root='data/cifar10_data/',
                              train=True,
                              transform=transforms.ToTensor(),
                              download=True)
cifar10_test = dsets.CIFAR10(root='data/cifar10_data/',
                              train=False,
                              transform=transforms.ToTensor(),
                              download=True)

Files already downloaded and verified
Files already downloaded and verified


In [111]:
data_loader = DataLoader(dataset=cifar10_train,
                       batch_size=batch_size,
                       shuffle=True,
                       drop_last=True)

In [112]:
data_test = DataLoader(dataset=cifar10_test,
                       batch_size=batch_size,
                       shuffle=True,
                       drop_last=True)

In [113]:
class CNN(nn.Module):
  def __init__(self):
        super(CNN, self).__init__()
        self.keep_prob = 0.5

        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 8, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer2 = nn.Sequential(
            nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        # self.layer3 = nn.Sequential(
        #     nn.Conv2d(16, 128, kernel_size=3, stride=1, padding=1),
        #     nn.ReLU(),
        #     nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc1 = nn.Linear(8*8*16, 64, bias=True)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 10)
        nn.init.uniform_(self.fc1.weight)

  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    # out = self.layer3(out)
    out = out.view(out.size(0), -1)
    out = self.fc1(out)
    out = self.fc2(out)
    out = self.fc3(out)
    return out


In [114]:
model = CNN().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [115]:
total_batch = len(data_loader)
print(total_batch)

1562


In [116]:
for epoch in range(training_epochs):
  avg_cost = 0

  for X, Y in data_loader:
    X=X.to(device)
    Y=Y.to(device)

    optimizer.zero_grad()
    y_hat=model(X)
    cost=criterion(y_hat, Y)

    cost.backward()
    optimizer.step()
    avg_cost+=cost/total_batch
  print(epoch, avg_cost)

0 tensor(2.0427, device='cuda:0', grad_fn=<AddBackward0>)
1 tensor(1.6021, device='cuda:0', grad_fn=<AddBackward0>)
2 tensor(1.4537, device='cuda:0', grad_fn=<AddBackward0>)
3 tensor(1.3780, device='cuda:0', grad_fn=<AddBackward0>)
4 tensor(1.3243, device='cuda:0', grad_fn=<AddBackward0>)
5 tensor(1.2768, device='cuda:0', grad_fn=<AddBackward0>)
6 tensor(1.2359, device='cuda:0', grad_fn=<AddBackward0>)
7 tensor(1.1992, device='cuda:0', grad_fn=<AddBackward0>)
8 tensor(1.1727, device='cuda:0', grad_fn=<AddBackward0>)
9 tensor(1.1447, device='cuda:0', grad_fn=<AddBackward0>)
10 tensor(1.1276, device='cuda:0', grad_fn=<AddBackward0>)
11 tensor(1.1048, device='cuda:0', grad_fn=<AddBackward0>)
12 tensor(1.0859, device='cuda:0', grad_fn=<AddBackward0>)
13 tensor(1.0720, device='cuda:0', grad_fn=<AddBackward0>)
14 tensor(1.0534, device='cuda:0', grad_fn=<AddBackward0>)
15 tensor(1.0412, device='cuda:0', grad_fn=<AddBackward0>)
16 tensor(1.0278, device='cuda:0', grad_fn=<AddBackward0>)
17 tens

In [124]:
accuracys = 0
with torch.no_grad():
  for X, Y in data_test:
    X = X.to(device)
    Y = Y.to(device)
    prediction = model(X)
    correct_prediction = torch.argmax(prediction, 1) == Y
    accuracy = correct_prediction.float().mean()
    print('Accuracy: ', accuracy.item())
    accuracys += accuracy

  print('Accuracy: %.3f' % ( 100. * accuracys / len(data_test)))

Accuracy:  0.59375
Accuracy:  0.6875
Accuracy:  0.5
Accuracy:  0.6875
Accuracy:  0.625
Accuracy:  0.46875
Accuracy:  0.65625
Accuracy:  0.71875
Accuracy:  0.71875
Accuracy:  0.65625
Accuracy:  0.71875
Accuracy:  0.71875
Accuracy:  0.75
Accuracy:  0.65625
Accuracy:  0.65625
Accuracy:  0.625
Accuracy:  0.625
Accuracy:  0.5625
Accuracy:  0.5625
Accuracy:  0.5625
Accuracy:  0.71875
Accuracy:  0.5
Accuracy:  0.6875
Accuracy:  0.75
Accuracy:  0.65625
Accuracy:  0.59375
Accuracy:  0.71875
Accuracy:  0.59375
Accuracy:  0.46875
Accuracy:  0.6875
Accuracy:  0.53125
Accuracy:  0.71875
Accuracy:  0.5625
Accuracy:  0.46875
Accuracy:  0.53125
Accuracy:  0.5625
Accuracy:  0.53125
Accuracy:  0.53125
Accuracy:  0.59375
Accuracy:  0.625
Accuracy:  0.5
Accuracy:  0.5625
Accuracy:  0.59375
Accuracy:  0.65625
Accuracy:  0.6875
Accuracy:  0.78125
Accuracy:  0.59375
Accuracy:  0.65625
Accuracy:  0.5625
Accuracy:  0.5
Accuracy:  0.625
Accuracy:  0.625
Accuracy:  0.5
Accuracy:  0.53125
Accuracy:  0.65625
Accur